# Couch.Potato Cockpit

IPython notebook for maintenance, debugging, live tweaks, and experimentation.

Use this for:
- Testing new prompts and workflows
- Debugging issues
- Quick one-off analysis
- Inspecting system state
- Patching scripts on the fly

In [ ]:
# Setup - run this first
import sys
sys.path.insert(0, '..')

from dotenv import load_dotenv
load_dotenv('../.env')

from brain import ask_claude, triage_alert, analyze_recon, parse_output, draft_report
from workflows.recon_triage import quick_triage
from workflows.alert_handler import handle_alert, batch_triage

print('Cockpit ready.')

## Quick Claude Query

In [ ]:
# Ask Claude anything
response = await ask_claude("What are the top 3 things to check for IDOR vulnerabilities?")
print(response)

## Triage an Alert

In [ ]:
# Triage a finding
alert = """
Found open redirect on login page:
https://example.com/login?next=https://evil.com
Redirects user after authentication to attacker-controlled site.
"""

result = await triage_alert(alert)
print(result)

## Analyze Recon Data

In [ ]:
# Analyze subdomains
subdomains = """
api.example.com
staging.example.com
admin.example.com
dev.example.com
mail.example.com
vpn.example.com
jenkins.example.com
grafana.example.com
"""

analysis = await analyze_recon(subdomains)
print(analysis)

## Parse Tool Output

In [ ]:
# Parse nmap output
nmap_output = """
Nmap scan report for target.com (192.168.1.1)
PORT     STATE SERVICE     VERSION
22/tcp   open  ssh         OpenSSH 8.2
80/tcp   open  http        nginx 1.18.0
443/tcp  open  ssl/http    nginx 1.18.0
3306/tcp open  mysql       MySQL 5.7.32
8080/tcp open  http-proxy  
9200/tcp open  elasticsearch Elasticsearch
"""

parsed = await parse_output(nmap_output, "nmap")
print(parsed)

## Draft a Report

In [ ]:
# Draft vulnerability report
report = await draft_report(
    vuln_type="IDOR",
    target="api.example.com",
    description="User can access other users' data by changing user_id parameter",
    impact="Unauthorized access to PII of all users",
    steps="1. Login as user A\n2. Go to /api/users/123/profile\n3. Change 123 to 124\n4. See user B's data"
)
print(report)

## System Status

In [ ]:
# Check system resources
import psutil
import platform

print(f"Host: {platform.node()}")
print(f"CPU: {psutil.cpu_percent()}%")
print(f"Memory: {psutil.virtual_memory().percent}%")
print(f"Disk: {psutil.disk_usage('/').percent}%")

## Batch Processing

In [ ]:
# Process multiple alerts at once
alerts = [
    "SQL injection in search: ' OR 1=1--",
    "XSS in comment field: <script>alert(1)</script>",
    "Exposed .git directory at /.git/config"
]

results = await batch_triage(alerts)
for i, result in enumerate(results):
    print(f"\n--- Alert {i+1} ---")
    print(f"Severity: {result.severity.value}")
    print(f"Valid: {result.is_valid}")
    print(result.raw_analysis[:500])

## Custom Prompt Testing

In [ ]:
# Test custom system prompts
import anthropic
import os

client = anthropic.AsyncAnthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

custom_system = """You are a bug bounty severity assessor. 
Rate findings on a scale of 1-10 with brief justification.
Consider: exploitability, impact, affected users."""

response = await client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=500,
    system=custom_system,
    messages=[{"role": "user", "content": "Rate: Reflected XSS in search parameter"}]
)

print(response.content[0].text)